# Venture Funding with Deep Learning

You work as a risk management associate at Alphabet Soup, a venture capital firm. Alphabet Soup’s business team receives many funding applications from startups every day. This team has asked you to help them create a model that predicts whether applicants will be successful if funded by Alphabet Soup.

The business team has given you a CSV containing more than 34,000 organizations that have received funding from Alphabet Soup over the years. With your knowledge of machine learning and neural networks, you decide to use the features in the provided dataset to create a binary classifier model that will predict whether an applicant will become a successful business. The CSV file contains a variety of information about these businesses, including whether or not they ultimately became successful.

In [ ]:
# Imports
import warnings
import pandas as pd
import tensorflow as tf
from pathlib import Path
from google.colab import files
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [ ]:
# Ignoring warnigns
warnings.filterwarnings('ignore')

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `applicants_data.csv` file into a Pandas DataFrame. Review the DataFrame, looking for categorical variables that will need to be encoded, as well as columns that could eventually define your features and target variables.  


In [ ]:
# Import applicants_data.csv to Google Colab
upload = files.upload()

Saving applicants_data.csv to applicants_data.csv


In [ ]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
applicant_data_df = pd.read_csv('applicants_data.csv')

# Review the DataFrame
applicant_data_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [ ]:
# Review the data types associated with the columns
applicant_data_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

### Step 2: Drop the “EIN” (Employer Identification Number) and “NAME” columns from the DataFrame, because they are not relevant to the binary classification model.

In [ ]:
# Drop the 'EIN' and 'NAME' columns from the DataFrame
applicant_data_df = applicant_data_df.drop(columns=['EIN','NAME'])

# Review the DataFrame
applicant_data_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


### Step 3: Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

In [ ]:
# Create a list of categorical variables 
categorical_variables = list(applicant_data_df.columns[applicant_data_df.dtypes == 'object'])

# Display the categorical variables list
categorical_variables

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [ ]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])

In [ ]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(encoded_data,columns=enc.get_feature_names(categorical_variables))

# Review the DataFrame
encoded_df

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Step 4: Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

> **Note** To complete this step, you will employ the Pandas `concat()` function that was introduced earlier in this course. 

In [ ]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df = pd.concat([encoded_df,applicant_data_df.drop(columns=categorical_variables)],axis=1)

# Review the DataFrame
encoded_df

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT,IS_SUCCESSFUL
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,108590,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,6692,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,0
34295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,0
34296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,0
34297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,1


### Step 5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 



In [ ]:
# Define the target set y using the IS_SUCCESSFUL column
# Define features set X by selecting all columns but IS_SUCCESSFUL
y,X = encoded_df.pop('IS_SUCCESSFUL'),encoded_df

# Display a sample of y
print(y.sample(5))
# Review the features DataFrame
X

8905     0
20281    1
27003    0
25331    0
11026    1
Name: IS_SUCCESSFUL, dtype: int64


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,108590
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,6692
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,142590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000
34295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000
34296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000
34297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000


### Step 6: Split the features and target sets into training and testing datasets.


In [ ]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)

### Step 7: Use scikit-learn's `StandardScaler` to scale the features data.

In [ ]:
# Create a StandardScaler instance
# Fit the scaler to the features training dataset
# Fit the scaler to the features training dataset
X_train_scaled,X_test_scaled = [StandardScaler().fit(X_train).transform(i) for i in [X_train,X_test]]

---

## Compile and Evaluate a Binary Classification Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [ ]:
# Custom Neural Network Class
class Custom_Neural_Network:
    """
        This class creates a custom neural network.
        The neural network is a Sequential model.
        The hidden layers are Dense layers that use the relu activation function.
        The output layer is a single node, Dense layer using the sigmoid activation function
        
        @PARAMS:
        - num_layers: integer value of number of hidden layers
        - number_input_features: integer value of the number of features within the featureset
        
        @METHODS:
        - __init__: Class constructor
        - on_init: Method to handle output and set variables
        - set_layers: Method to set the number of nodes for each hidden layer 
        - print_layers: Method to print layers variable values
        - set_neural_network_layers: Method to set layers for neural network
        - get_neural_network: Method to return neural network
        
    """
    
    # Class constuctor
    def __init__(self, number_layers, number_input_features):
        
        """
            Initializes variables and calls on_init method
        """
        
        # Setting Class Variables
        self.number_layers = number_layers
        self.number_input_features = number_input_features
        self.current_layer = number_input_features
        self.number_output_neurons = 1
        self.layers = []
        self.relu = 'relu'
        self.sigmoid = 'sigmoid'
        self.nn = Sequential()
        self.on_init()
        
        # Method to handle output and set variable values
    def on_init(self):
        
        """
            Handles output and sets variable values
        """
        # Printing the number of input features
        print(self.number_input_features)
        # Setting layers
        self.set_layers()
        # Printing layers
        self.print_layers()
        # Setting neural network layers
        self.set_neural_network_layers()
        # Displaying neural network summary
        display(self.nn.summary())
        
        # Method to set the number of nodes for each hidden layer
    def set_layers(self):
        
        """
            Sets the number of nodes for each hidden layer
        """
        
        # Looping over number of layers
        for i in range(self.number_layers):
            # Setting current layer values
            self.current_layer = (self.current_layer + 1) // 2
            # Appending current layer to layers list
            self.layers.append(self.current_layer)
                
        # Method to print layer values        
    def print_layers(self):
        
        """
            Number of nodes for each hidden layer
        """
        # Looping over layer values
        for i in self.layers:
            # Printing layer value
            print(i)
            
        # Method to set the layers for the neural network    
    def set_neural_network_layers(self):
        
        """
            Adds the layers of the neural network
        """
        # Creating the first layer
        self.nn.add(Dense(self.layers[0], input_dim=self.number_input_features, activation=self.relu))
        
        # Looping over a range of the layers
        for i in range(1,len(self.layers)):
            # Adding each hidden layer
            self.nn.add(Dense(self.layers[i], activation=self.relu))
        # Adding the final output layer
        self.nn.add(Dense(self.number_output_neurons, activation=self.sigmoid))
        
        # Method to return neural network
    def get_neural_network(self):
        
        """
            Returns the neural network itself
        """
        # Returning the neural network
        return self.nn

In [ ]:
# Define the number of features
# Review the number of features
# Define the number of neurons in the output layer
# Define the number of hidden nodes for the first hidden layer
# Review the number of hidden nodes in the first layer
# Define the number of hidden nodes for the second hidden layer
# Review the number of hidden nodes in the second layer
# Create the Sequential model instance
# Add the first hidden layer
# Add the second hidden layer
# Add the output layer to the model specifying the number of output neurons and activation function
# Display the Sequential model summary
nn = Custom_Neural_Network(2,X.columns.size).get_neural_network()

116
58
29
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 58)                6786      
                                                                 
 dense_1 (Dense)             (None, 29)                1711      
                                                                 
 dense_2 (Dense)             (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


None

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [ ]:
# Utility method to compile and fit model
def compile_fit_model(nn, data, epochs):
    # Compile neural network
    nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit neural newtwork
    nn.fit(data,y_train,epochs=epochs)

In [ ]:
# Compile the Sequential model
# Fit the model using 50 epochs and the training data
compile_fit_model(nn,X_train_scaled,50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5795 - accuracy: 0.7173
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7306
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7307
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7309
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7312
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accuracy: 0.7315
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5450 - accuracy: 0.7340
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5439 - accuracy: 0.7330
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5429 - accuracy: 0.7350
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5427 - accuracy: 0.7347

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [ ]:
# Method to print evaluation statistics
def evaluate_model(nn, data, title=''):
    # Print title
    print(title)
    
    # Extract loss and accuracy statistics
    loss, accuracy = nn.evaluate(data,y_test,verbose=2)

    # Display the model loss and accuracy results
    print(f"Loss: {loss}, Accuracy: {accuracy}")

In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
evaluate_model(nn,X_test_scaled)


268/268 - 0s - loss: 0.5576 - accuracy: 0.7303 - 481ms/epoch - 2ms/step
Loss: 0.5576133728027344, Accuracy: 0.7302623987197876


### Step 4: Save and export your model to an HDF5 file, and name the file `AlphabetSoup.h5`. 


In [ ]:
# Utility method to save model
def save_model(nn, extension=''):
    # Set file path
    file_path = f'AlphabetSoup{extension}.h5'
    # Save model
    nn.save(file_path)
    # Download file
    files.download(file_path)

In [ ]:
# Set the model's file path
# Export your model to a HDF5 file
save_model(nn)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---

## Optimize the neural network model


### Step 1: Define at least three new deep neural network models (resulting in the original plus 3 optimization attempts). With each, try to improve on your first model’s predictive accuracy.

> **Rewind** Recall that perfect accuracy has a value of 1, so accuracy improves as its value moves closer to 1. To optimize your model for a predictive accuracy as close to 1 as possible, you can use any or all of the following techniques:
>
> * Adjust the input data by dropping different features columns to ensure that no variables or outliers confuse the model.
>
> * Add more neurons (nodes) to a hidden layer.
>
> * Add more hidden layers.
>
> * Use different activation functions for the hidden layers.
>
> * Add to or reduce the number of epochs in the training regimen.


### Alternative Model 1

In [ ]:
# Alternative_Features class
class Alternative_Features:
    
    """
    This class processes and returns alternative feature data.

    @METHODS:
    - __init__: Class constructor
    - on_init: Method to handle logical steps
    - set_columns_to_drop: Method to set value of columns to drop
    - get_features: Method to get DataFrame of features
    - set_alt_features: Method to set the alternative features values
    - set_scaler: Method to set the value of the scaler
    - transform_features: Method to scale features
    - set_scaled_features: Method to set scaled feautre data
    - get_feature_data: Method to return feature data
    """


    # Class constructor
    def __init__(self, X, X_train, X_test):

        """
            Initialized variables and calls step logic method
        """

        # Class variables
        self.original_columns_names = ['SPECIAL_CONSIDERATIONS', 'STATUS', 'USE_CASE']
        self.columns_to_drop = []
        self.X = X
        self.X_train = X_train
        self.X_test = X_test
        self.X_alt_scaler = None
        self.X_train_alt = None
        self.X_test_alt = None
        self.X_train_scaled_alt = None
        self.X_test_scaled_alt = None
        self.on_init()

        # Method to handle step logic
    def on_init(self):

        """
            Handles step logic
        """

        # Sets columns to drop
        self.set_columns_to_drop()
        # Sets alternative features
        self.set_alt_features()
        # Sets scaler
        self.set_scaler()
        # Sets scaled features
        self.set_scaled_features()

        # Method to set columns to drop
    def set_columns_to_drop(self):

        """
            Sets columns to drop
        """

        # Loop over original columns names
        for i in self.original_columns_names:
            # Get feature names
            for feature_name in self.X.columns[pd.Series(self.X.columns).str.startswith(i)]:
                # Add feature name to columns to drop
                self.columns_to_drop.append(feature_name)

       # Method to get features
    def get_features(self,features):

        """
           Gets features
        """
        # Return features
        return features.copy().drop(self.columns_to_drop, axis=1)

        # Method to set alternative features
    def set_alt_features(self):

        """
            Sets alternative features
        """

        # Set alternative training features
        self.X_train_alt = self.get_features(self.X_train)
        # Set alternative test features
        self.X_test_alt = self.get_features(self.X_test)

        # Method to set scaler value
    def set_scaler(self):

        """
            Sets scaler
        """

        # Set value for scaler
        self.X_alt_scaler = StandardScaler().fit(self.X_train_alt)

        # Method to return and transform feature
    def transform_features(self, features):

        """
            Transforms feature
        """

        # Return/transform feature
        return self.X_alt_scaler.transform(features)

        # Method to set scaled features
    def set_scaled_features(self):

        """
            Sets scaled feature data
        """

        # Set scaled training features
        self.X_train_scaled_alt = self.transform_features(self.X_train_alt)
        # Set scaled test features
        self.X_test_scaled_alt = self.transform_features(self.X_test_alt)

        # Method to return feature data
    def get_feature_data(self):

        """
            Returns feature data
        """

        # Returning feature data
        return self.X_train_scaled_alt, self.X_test_scaled_alt, self.X_test_alt

In [ ]:
# Extracting X_train_scaled_alt, X_test_scaled_alt, and X_test_alt variables from Alternative_Features class
X_train_scaled_alt,X_test_scaled_alt,X_test_alt =  Alternative_Features(X, X_train, X_test).get_feature_data()

In [ ]:
# Define the number of features
# Review the number of features
# Define the number of neurons in the output layer
# Define the number of hidden nodes for the first hidden layer
# Review the number of hidden nodes in the first layer
# Define the number of hidden nodes for the second hidden layer
# Review the number of hidden nodes in the second layer
# Define the number of hidden nodes for the third hidden layer
# Review the number of hidden nodes in the third layer
# Define the number of hidden nodes for the fourth hidden layer
# Review the number of hidden nodes in the fourth layer
# Create the Sequential model instance
# First hidden layer
# Seond layer
# Third layer
# Foruth layer
# Output layer
# Check the structure of the model
nn_A1 = Custom_Neural_Network(4,X_test_alt.columns.size).get_neural_network()

108
54
27
14
7
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 54)                5886      
                                                                 
 dense_4 (Dense)             (None, 27)                1485      
                                                                 
 dense_5 (Dense)             (None, 14)                392       
                                                                 
 dense_6 (Dense)             (None, 7)                 105       
                                                                 
 dense_7 (Dense)             (None, 1)                 8         
                                                                 
Total params: 7,876
Trainable params: 7,876
Non-trainable params: 0
_________________________________________________________________


None

In [ ]:
# Compile and Fit the model
compile_fit_model(nn_A1,X_train_scaled_alt,35)

Epoch 1/35
804/804 [==============================] - 3s 2ms/step - loss: 0.5732 - accuracy: 0.7176
Epoch 2/35
804/804 [==============================] - 2s 3ms/step - loss: 0.5523 - accuracy: 0.7306
Epoch 3/35
804/804 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7301
Epoch 4/35
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7310
Epoch 5/35
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7323
Epoch 6/35
804/804 [==============================] - 2s 2ms/step - loss: 0.5476 - accuracy: 0.7321
Epoch 7/35
804/804 [==============================] - 2s 2ms/step - loss: 0.5469 - accuracy: 0.7329
Epoch 8/35
804/804 [==============================] - 2s 2ms/step - loss: 0.5456 - accuracy: 0.7339
Epoch 9/35
804/804 [==============================] - 3s 4ms/step - loss: 0.5456 - accuracy: 0.7335
Epoch 10/35
804/804 [==============================] - 2s 3ms/step - loss: 0.5456 - accuracy: 0.7329

#### Alternative Model 2

In [ ]:
# Define the number of features
# Review the number of features
# Define the number of neurons in the output layer
# Define the number of hidden nodes for the first hidden layer
# Review the number of hidden nodes in the first layer
# Define the number of hidden nodes for the second hidden layer
# Review the number of hidden nodes in the second layer
# Define the number of hidden nodes for the third hidden layer
# Review the number of hidden nodes in the third layer
# Create the Sequential model instance
# First hidden layer
# Second hidden layer
# Third hidden layer
# Output layer
# Check the structure of the model
nn_A2 = Custom_Neural_Network(3,X_test_alt.columns.size).get_neural_network()

108
54
27
14
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 54)                5886      
                                                                 
 dense_9 (Dense)             (None, 27)                1485      
                                                                 
 dense_10 (Dense)            (None, 14)                392       
                                                                 
 dense_11 (Dense)            (None, 1)                 15        
                                                                 
Total params: 7,778
Trainable params: 7,778
Non-trainable params: 0
_________________________________________________________________


None

In [ ]:
# Compile and Fit the model
compile_fit_model(nn_A2,X_train_scaled_alt,25)

Epoch 1/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5733 - accuracy: 0.7194
Epoch 2/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5531 - accuracy: 0.7296
Epoch 3/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5504 - accuracy: 0.7304
Epoch 4/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.7317
Epoch 5/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5473 - accuracy: 0.7334
Epoch 6/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7331
Epoch 7/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5466 - accuracy: 0.7335
Epoch 8/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5461 - accuracy: 0.7334
Epoch 9/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5453 - accuracy: 0.7340
Epoch 10/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5450 - accuracy: 0.7341

### Step 2: After finishing your models, display the accuracy scores achieved by each model, and compare the results.

In [ ]:
# Utility method to get range from models list
models_range = lambda models: range(len(models))

In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
# Display the model loss and accuracy results
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
# Display the model loss and accuracy results
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
# Display the model loss and accuracy results

# Variable to hold models
models = [nn,nn_A1,nn_A2]

# Loop over range of models length
for i in models_range(models):
    # Set variable for scaled data
    scaled_data = X_test_scaled_alt if i else X_test_scaled
    # Set title
    title = f'Alternative Model {i} Results' if i else 'Original Model Results'
    # Print model evaluation
    evaluate_model(models[i],scaled_data,'\n' + title)


Original Model Results
268/268 - 0s - loss: 0.5576 - accuracy: 0.7303 - 379ms/epoch - 1ms/step
Loss: 0.5576133728027344, Accuracy: 0.7302623987197876

Alternative Model 1 Results
268/268 - 1s - loss: 0.5554 - accuracy: 0.7322 - 509ms/epoch - 2ms/step
Loss: 0.5553603172302246, Accuracy: 0.7322449088096619

Alternative Model 2 Results
268/268 - 0s - loss: 0.5552 - accuracy: 0.7310 - 446ms/epoch - 2ms/step
Loss: 0.5552314519882202, Accuracy: 0.7309620976448059


### Step 3: Save each of your alternative models as an HDF5 file.


In [ ]:
# Set the file path for the second alternative model
# Export your model to a HDF5 file
# Set the file path for the first alternative model
# Export your model to a HDF5 file

# Loop over range of models length
for i in models_range(models[1:]):
    # Save model
    save_model(models[i], f'_A{i+1}')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>